# 임포트

In [1]:
import os
import langchain_chroma
import openai
from openai import OpenAI
import chromadb
import fitz  # PDF에서 텍스트 추출
from dotenv import load_dotenv
from fastapi import FastAPI, UploadFile, File
from sqlalchemy import create_engine, text
from sqlalchemy.ext.declarative import declarative_base
import pandas as pd
import psycopg2
import torch
from sklearn.metrics.pairwise import cosine_similarity
from langchain_text_splitters import RecursiveCharacterTextSplitter


# .env 파일 로드
load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
POST_DB_HOST = os.getenv("POST_DB_HOST")
POST_DB_NAME = os.getenv("POST_DB_NAME")
POST_DB_USER = os.getenv("POST_DB_USER")
POST_DB_PASSWD = os.getenv("POST_DB_PASSWD")
POST_DB_PORT = os.getenv("POST_DB_PORT")
# PostgreSQL 연결 엔진 생성
post_engine = create_engine(f'postgresql://{POST_DB_USER}:{POST_DB_PASSWD}@{POST_DB_HOST}:{POST_DB_PORT}/{POST_DB_NAME}')
db = psycopg2.connect(host=POST_DB_HOST, dbname=POST_DB_NAME,user=POST_DB_USER,password=POST_DB_PASSWD,port=POST_DB_PORT)
cursor = db.cursor()
db.autocommit = False

In [12]:
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings
from langchain.schema import Document
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.schema.runnable import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from getpass import getpass
import os

# 데이터 불러오기

In [18]:
# 이력서 불러오기
path = "./data/빅데이터AI_이력서.pdf"
doc = fitz.open(path)
for page in doc:
    resume_text = page.get_text()
    print(resume_text)

📄 이력서 (Resume)
[기본 정보]
이름: 홍길동
연락처: 010-1234-5678
이메일: honggildong.ai@gmail.com
주소: 서울특별시 강남구 테헤란로 123
[학력]
고려대학교 컴퓨터학과 졸업 (2018.03 ~ 2024.02)
GPA: 3.85 / 4.5
관련 과목: 머신러닝, 데이터마이닝, 통계학, 빅데이터처리, 딥러닝 이론과 실습
[기술 스택]
Programming: Python, SQL, R
Frameworks/Libraries: Scikit-learn, TensorFlow, PyTorch, Pandas, NumPy
Tools: Jupyter, Git, Docker, Tableau
DBMS: MySQL, MongoDB, Hadoop(HDFS), Spark
Cloud: Google Colab, AWS EC2 & S3 (기초 수준)
[프로젝트 경험]
1. 신문 기사 기반 감성 분석 모델 개발 (2023.03 ~ 2023.06)
자연어처리(NLP) 기반 감성 분류 모델 개발
KoNLPy와 Scikit-learn을 이용한 전처리 및 모델 학습
정확도 86% 달성
2. 머신러닝 기반 개인 맞춤형 영화 추천 시스템 (2023.09 ~ 2023.12)
📄 이력서 (Resume)
1

Content-based Filtering 및 Collaborative Filtering 기법 적용
Streamlit으로 웹 인터페이스 구현
kaggle 데이터셋 기반, Precision@10: 0.73
[자격증]
ADsP (데이터분석 준전문가) – 2023.08
SQLD (SQL 개발자) – 2024.01
📄 이력서 (Resume)
2



In [ ]:
df = pd.read_csv("../data_backup/rec_data.csv")

In [3]:
chroma_client = chromadb.Client()
collection_name = 'chroma_test'
try:
    collection = chroma_client.get_collection(name=collection_name,
                                              metadata={"hnsw:space": "cosine"})
except:
    collection = chroma_client.create_collection(name=collection_name)

In [4]:
vectordb = Chroma(
    collection_name="chroma_test",  # ✅ 기존 컬렉션 이름
    persist_directory="./chroma_data"   
)

In [14]:
documents = []
for _, row in df.iterrows():
    text = row["chunk"]
    metadata = row.drop(["chunk"]).to_dict()
    documents.append(Document(page_content=text, metadata=metadata))

In [15]:
embedding_model = OpenAIEmbeddings(model="text-embedding-3-small")

vectordb = Chroma.from_documents(
    documents=documents,
    embedding=embedding_model,
    persist_directory="./chroma_data",  # 저장 경로
    collection_name="chroma_test"
)

vectordb.persist()

/tmp/ipykernel_250343/2890479165.py:1: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embedding_model = OpenAIEmbeddings(model="text-embedding-3-small")


RateLimitError: Error code: 429 - {'error': {'message': 'Request too large for text-embedding-3-small in organization org-IwGpHR0tmQinAuiSLBSiKYgU on tokens per min (TPM): Limit 1000000, Requested 1034038. The input or output tokens must be reduced in order to run successfully. Visit https://platform.openai.com/account/rate-limits to learn more.', 'type': 'tokens', 'param': None, 'code': 'rate_limit_exceeded'}}

In [16]:
# 위 코드가 분당 토큰 100만을 넘겼기에 사용
from time import sleep

batch_size = 500
batch_documents = [documents[i:i+batch_size] for i in range(0, len(documents), batch_size)]

embedding_model = OpenAIEmbeddings(model="text-embedding-3-small")

vectordb = None

for i, batch in enumerate(batch_documents):
    print(f"▶ Processing batch {i+1}/{len(batch_documents)}...")
    if i == 0:
        vectordb = Chroma.from_documents(
            documents=batch,
            embedding=embedding_model,
            persist_directory="./chroma_data",
            collection_name="chroma_test"
        )
    else:
        vectordb.add_documents(batch)

    vectordb.persist()
    sleep(65)

▶ Processing batch 1/28...


/tmp/ipykernel_250343/1925841830.py:23: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  vectordb.persist()


▶ Processing batch 2/28...
▶ Processing batch 3/28...
▶ Processing batch 4/28...
▶ Processing batch 5/28...
▶ Processing batch 6/28...
▶ Processing batch 7/28...
▶ Processing batch 8/28...
▶ Processing batch 9/28...
▶ Processing batch 10/28...
▶ Processing batch 11/28...
▶ Processing batch 12/28...
▶ Processing batch 13/28...
▶ Processing batch 14/28...
▶ Processing batch 15/28...
▶ Processing batch 16/28...
▶ Processing batch 17/28...
▶ Processing batch 18/28...
▶ Processing batch 19/28...
▶ Processing batch 20/28...
▶ Processing batch 21/28...
▶ Processing batch 22/28...
▶ Processing batch 23/28...
▶ Processing batch 24/28...
▶ Processing batch 25/28...
▶ Processing batch 26/28...
▶ Processing batch 27/28...
▶ Processing batch 28/28...


In [74]:
vectordb = Chroma(
    collection_name="chroma_test",
    persist_directory="./chroma_data",
    embedding_function=embedding_model
)
retriever = vectordb.as_retriever(search_type="similarity", search_kwargs={"k": 5})

In [81]:
# 프롬프트 템플릿
prompt = ChatPromptTemplate.from_template("""
당신은 인재매칭 AI 어시스턴트입니다. 사용자 이력서에 기반하여 채용공고를 5개 추천해주세요.

- 반드시 이력서에 기반할 것.
- 채용공고 상세는 반드시 모두 출력할 것.
- 출력 시 채용공고의 양식을 사용할 것.
- 채용공고 추천 이유를 한줄로 설명할 것.                            
- 채용공고 전체 내용을 기반하여 분석할 것. 제목만 보고 판단하지 말 것.
- page_content의 전체 텍스트를 기준으로 판단할 것.

### 출력 형식 예시
- 직무명 :
- 공고 번호:                           
- 산업 분야 : 
- 필요 기술 : 
- 채용공고 상세 :
- 추천 사유 :

이 정보를 바탕으로 사용자 정보에 가장 적합한 채용 공고 5개를 추천해주세요.
출력 형식과 동일한 형식으로 출력해주세요.                                          

채용공고:
{context}

이력서:
{question}
""")

# RAG Chain
rag_chain1 = (
    {"context": retriever | (lambda docs: "\n\n".join(
            f"{doc.metadata.get('rec_idx')}\n{doc.metadata.get('page')} \n{doc.page_content.strip()}"
            for doc in docs
        )),
     "question": RunnablePassthrough()}
    | prompt
    | ChatOpenAI(model="gpt-4o", temperature=0.3)
    | StrOutputParser()
)


In [78]:
doc

Document(metadata={'Unnamed: 0': 4269, 'career': '경력무관 · 정규직', 'company_nm': '(주)소만사', 'company_place': '경기 성남시 수정구', 'education': '대학(2,3년)↑', 'rec_idx': 50215560, 'recruit_kewdcdnm': "['정보보안', '솔루션', 'DLP', 'C++', 'C언어']"}, page_content='네트워크 보안 SW 개발자(C++) 채용\n\n채용공고 상세')

In [ ]:
query = resume_text
answer1 = rag_chain1.invoke(query)
print(answer1)

사용자의 이력서를 기반으로 적합한 채용 공고 5개를 추천드리겠습니다. 사용자는 데이터 분석 및 개발 관련 경험이 있으며, 관련 자격증을 보유하고 있습니다. 이러한 정보를 바탕으로 적합한 채용 공고를 선택했습니다.

1. 
- 직무명 : 데이터 분석가
- 공고 번호: 50226095
- 산업 분야 : IT개발·데이터
- 필요 기술 : 데이터 분석, SQL, 머신러닝
- 채용공고 상세 : ㈜LG CNS에서 데이터 분석가를 모집하며, 데이터 분석 및 머신러닝 관련 프로젝트 경험이 요구됩니다.
- 추천 사유 : 사용자의 데이터 분석 및 SQL 관련 자격증과 프로젝트 경험이 직무 요구사항과 일치합니다.

2. 
- 직무명 : 데이터 엔지니어
- 공고 번호: 50226095
- 산업 분야 : IT개발·데이터
- 필요 기술 : 데이터 엔지니어링, SQL, Python
- 채용공고 상세 : 데이터 엔지니어링 및 데이터 파이프라인 구축 경험이 있는 인재를 찾고 있습니다.
- 추천 사유 : 사용자의 SQL 및 Python 경험이 데이터 엔지니어링 직무에 적합합니다.

3. 
- 직무명 : 머신러닝 엔지니어
- 공고 번호: 50226095
- 산업 분야 : IT개발·데이터
- 필요 기술 : 머신러닝, Python, TensorFlow
- 채용공고 상세 : 머신러닝 모델 개발 및 최적화 경험이 필요하며, TensorFlow 사용 경험이 요구됩니다.
- 추천 사유 : 사용자의 머신러닝 프로젝트 경험과 Python 기술이 직무에 적합합니다.

4. 
- 직무명 : 데이터 사이언티스트
- 공고 번호: 50226095
- 산업 분야 : IT개발·데이터
- 필요 기술 : 데이터 분석, 머신러닝, 데이터 시각화
- 채용공고 상세 : 데이터 분석 및 시각화, 머신러닝 모델링 경험이 있는 인재를 모집합니다.
- 추천 사유 : 사용자의 데이터 분석 및 머신러닝 경험이 직무 요구사항과 부합합니다.

5. 
- 직무명 : SQL 개발자
- 공고 번호: 50226095
- 산업 분야 : IT개발·데이터